# 거리계산 모듈화

In [1]:
import bike_distance_calc

In [2]:
lat = 37.55564880
long = 126.91062927

In [3]:
f = bike_distance_calc.bike_distance(lat,long)

In [5]:
f.result

[{'rackTotCnt': 15,
  'stationName': '102. 망원역 1번출구 앞',
  'parkingBikeTotCnt': 3,
  'shared': 20,
  'stationLatitude': 37.5556488,
  'stationLongitude': 126.91062927,
  'stationId': 'ST-4',
  'distance': 0.0},
 {'rackTotCnt': 14,
  'stationName': '103. 망원역 2번출구 앞',
  'parkingBikeTotCnt': 1,
  'shared': 7,
  'stationLatitude': 37.55495071,
  'stationLongitude': 126.91083527,
  'stationId': 'ST-5',
  'distance': 79.7199476062},
 {'rackTotCnt': 9,
  'stationName': '184. 7번가피자 서교망원점',
  'parkingBikeTotCnt': 1,
  'shared': 11,
  'stationLatitude': 37.55894852,
  'stationLongitude': 126.90775299,
  'stationId': 'ST-342',
  'distance': 445.9914630036},
 {'rackTotCnt': 25,
  'stationName': '497.합정동주민센터 앞',
  'parkingBikeTotCnt': 20,
  'shared': 80,
  'stationLatitude': 37.551754,
  'stationLongitude': 126.91230011,
  'stationId': 'ST-2156',
  'distance': 457.4439852306},
 {'rackTotCnt': 13,
  'stationName': '104. 합정역 1번출구 앞',
  'parkingBikeTotCnt': 17,
  'shared': 131,
  'stationLatitude': 37.

# 주소 -> 위,경도 (kakao restapi)

In [6]:
import requests
import json

In [7]:
address = '서울 중구 회현동1가'

In [9]:
def changeAddress(address):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    headers = {"Authorization": "KakaoAK ddade78155e5ff37aab8ddc00f07c0df"} #kakao rest api key
    address_json = requests.get(url, headers=headers).json()['documents'][0]
    return address_json['x'],address_json['y']

In [10]:
changeAddress(address)

('126.981204133005', '37.5555892070291')

# 서울 공원 및 문화공간 데이터 수집 및 가공
- https://data.seoul.go.kr/dataList/OA-15487/S/1/datasetView.do
- https://data.seoul.go.kr/dataList/OA-394/S/1/datasetView.do

In [20]:
import requests
import pandas as pd

In [435]:
datas = ['SearchParkInfoService','culturalSpaceInfo']

In [443]:
url = 'http://openapi.seoul.go.kr:8088/4e465a626d30363134304465454b78/json/culturalSpaceInfo/1/1000/'   #문화공간
req = requests.get(url).json()
jsons = req['culturalSpaceInfo']['row']
culture = pd.DataFrame(jsons)
culture = culture[['FAC_NAME','X_COORD','Y_COORD']]
culture = culture.rename({'FAC_NAME':'name','X_COORD':'latitude','Y_COORD':'longtitude'},axis = 1)

In [475]:
url = 'http://openAPI.seoul.go.kr:8088/4e465a626d30363134304465454b78/json/SearchParkInfoService/1/1000/'  #공원
req = requests.get(url).json()
jsons = req['SearchParkInfoService']['row']
park = pd.DataFrame(jsons)
park = park[['P_PARK','LONGITUDE','LATITUDE']]
park = park.rename({'P_PARK':'name','LONGITUDE':'longtitude','LATITUDE':'latitude'},axis = 1)

In [463]:
# culture.to_csv('서울 문화공간.csv',index = False)

In [476]:
# park.to_csv('서울 공원.csv',index = False)

# 문화재 데이터 수집 및 가공
- https://www.cha.go.kr/html/HtmlPage.do?pg=/publicinfo/pbinfo3_0202.jsp&mn=NS_04_04_03

In [263]:
import json
import xmltodict
# !pip3 install xmltodict

In [387]:
co = ['11','12','13','14','15','16','17','18','21','22','23','24','25','31','79','80']
page_index = list(range(1,9))
params = {
    'ccbaCtcd':'11',
    'pageUnit':100}

In [411]:
temp = []
for i in co:
    for page in page_index:
        url = f'http://www.cha.go.kr/cha/SearchKindOpenapiList.do?ccbaKdcd={i}&pageIndex={page}'
        req = requests.get(url,params=params)        
        s = xmltodict.parse(req.text)['result']
        if len(s.keys()) == 4:
            if str(type(s['item'])) == "<class 'dict'>":
                s['item'] = [s['item']]
            temp.append(s['item'])
        else:
            break;

In [471]:
heritage = sum(temp,[])
heritage = pd.DataFrame(heritage)
heritage = heritage[['ccmaName','ccbaMnm1','longitude','latitude']]
heritage = heritage.rename({'longitude':'longtitude'},axis = 1)
nolat = heritage[heritage['latitude']=='0'].index
heritage = heritage.drop(nolat,axis = 0).reset_index(drop = True)

In [ ]:
# heritage.to_csv('서울 문화재.csv',index = False)